In [ ]:
import multiprocessing as mp
import random
from pathlib import Path

import numpy as np
import torch
import torch.nn.functional as F

from attacker import LinfPGDAttack
from data import mnist, fashion_mnist
from models import LatentClf
from simulation.make_data import get_dataloader
from training import OnManifoldPerturbation
from training.vae_train import ClassifierTrainer, VAETrainer
from utils import load_data, load_checkpoint, concatenate, save_data
import pandas as pd

random_seed = 999

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


def generate_adv(net, test_loader, device, epsilon=1.0, alpha=0.01, k=7):
    net.eval()
    adv = LinfPGDAttack(net, epsilon=epsilon, alpha=alpha, k=k)
    
    advs = None
    correct_ys = None
    total = None
    total_ys = None
    perturbs = None
    for X, y in test_loader:
        adv_x = adv.perturb(X.to(device), y.long().to(device))
        perturbs = concatenate(perturbs, (adv_x - X.to(device)).detach().cpu().numpy())
        out = net(adv_x)
        out = F.softmax(out, dim=1)
        _, predicted = out.max(1)
        idx = ~predicted.eq(y.to(device))
        advs = concatenate(advs, adv_x[idx].detach().cpu().numpy())
        correct_ys = concatenate(correct_ys, y[idx].detach().cpu().numpy())
        total = concatenate(total, adv_x.detach().cpu().numpy())
        total_ys = concatenate(total_ys, y.detach().cpu().numpy())
        
    adv_dataloader = get_dataloader(advs, correct_ys, drop_last=False, shuffle=False)
    total_adv_dataloader = get_dataloader(total, total_ys, drop_last=False, shuffle=False)
    
    return adv_dataloader, perturbs, advs, correct_ys, total_adv_dataloader


def accuracy2(net, test_loader, device):
    net.eval()
    correct = 0
    total_set = 0

    for X, y in test_loader:
        X = X.to(device)
        out = net(X)
        out = F.softmax(out, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        correct += predicted.eq(y.to(device)).sum().item()

        total_set += X.size(0)

    benign_acc = correct / total_set
    print('accuracy: {0}'.format(benign_acc))
    return benign_acc, correct, total_set


def accuracy3(net, test_loader, device, epsilon=1.0, alpha=0.01, k=7):
    net.eval()
    correct = 0
    total_set = 0
    
    adv = LinfPGDAttack(net, epsilon=epsilon, alpha=alpha, k=k)

    for X, y in test_loader:
        adv_x = adv.perturb(X.to(device), y.long().to(device))
        
        out = net(adv_x)
        out = F.softmax(out, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        correct += predicted.eq(y.to(device)).sum().item()

        total_set += X.size(0)

    benign_acc = correct / total_set
    print('accuracy: {0}'.format(benign_acc))
    return benign_acc, correct, total_set


def accuracy(net, test_loader, device, epsilon=1.0, alpha=0.01, k=7):
    net.eval()
    benign_correct = 0
    adv_correct = 0
    total_set = 0
    correct_list = None
    correct_label = None
    adv_list = None
    benign_softmax_list = None
    correct_softmax_list = None
    adversarial_softmax_list = None
    correct_adversarial_softmax_list = None

    adv = LinfPGDAttack(net, epsilon=epsilon, alpha=alpha, k=k)

    for X, y in test_loader:
        X = X.to(device)
        out = net(X)
        out = F.softmax(out, dim=1)
        benign_softmax_list = concatenate(benign_softmax_list, out.detach().cpu().numpy())
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        benign_correct += predicted.eq(y.to(device)).sum().item()

        correct_softmax_list = concatenate(correct_softmax_list, out[idx].detach().cpu().numpy())
        correct_list = concatenate(correct_list, X[idx].detach().cpu().numpy())
        correct_label = concatenate(correct_label, y[idx].detach().cpu().numpy())

        adv_x = adv.perturb(X.to(device), y.long().to(device))

        perturbation = adv_x - X
        out = net(adv_x)
        out = F.softmax(out, dim=1)
        adversarial_softmax_list = concatenate(adversarial_softmax_list, out.detach().cpu().numpy())
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        correct_adversarial_softmax_list = concatenate(correct_adversarial_softmax_list, out[idx].detach().cpu().numpy())
        adv_correct += predicted.eq(y.to(device)).sum().item()

        adv_list = concatenate(adv_list, perturbation.detach().cpu().numpy())

        total_set += X.size(0)

    benign_acc = benign_correct / total_set
    adv_acc = adv_correct / total_set
    print('benign accuracy: {0}\tadversarial accuracy: {1}'.format(benign_acc, adv_acc))
    return benign_acc, adv_acc, correct_list, correct_label, adv_list, correct_softmax_list, correct_adversarial_softmax_list, benign_softmax_list, adversarial_softmax_list


def make_perturbed_data(classifier, gan, l_clf, loader, device, epsilon=0.3, k=7, alpha=0.01, filter_=False):
    on_adv = OnManifoldPerturbation(classifier, gan, device, eta=epsilon, k=k, alpha=alpha)

    on_adv_Xs = None
    on_ys = None
    on_zs = None
    on_original_zs = None
    attack_succ_idxs = None
    orig_X = None
    orig_y = None

    print('creating perturbed data..')

    for X, y in loader:
        #     X = X.reshape((X.shape[0], -1))
        z, z_pert, adv_x = on_adv.perturb(X.to(device), y.long().to(device))
        on_adv_Xs = concatenate(on_adv_Xs, adv_x.detach().cpu().numpy())
        on_zs = concatenate(on_zs, z_pert.detach().cpu().numpy())
        on_ys = concatenate(on_ys, y.detach().cpu().numpy())
        on_original_zs = concatenate(on_original_zs, z.detach().cpu().numpy())
        orig_X = concatenate(orig_X, X.detach().numpy())
        orig_y = concatenate(orig_y, y.detach().numpy())

        f_x = classifier(adv_x)
        f_x = F.softmax(f_x, dim=1)
        _, predicted = f_x.max(1)

        idxs = ~predicted.eq(y.to(device))
        attack_succ_idxs = concatenate(attack_succ_idxs, idxs.detach().cpu().numpy())


    adv_loader = get_dataloader(on_adv_Xs, on_ys)
    z_loader = get_dataloader(on_zs, on_ys)

    on_xs = None
    on_ys = None
    on_preds = None

    print('filtering perturbed data..')

    for x_tilde, y in adv_loader:
        out, _ = gan.encoder(x_tilde.to(device))
        out = l_clf(out.unsqueeze(1))
        out = F.softmax(out, dim=1)
        _, pred = out.max(1)

        idx = pred.eq(y.to(device))

        on_adv_filtered = x_tilde[idx].detach().cpu().numpy()
        on_adv_filtered_label = y[idx].detach().cpu().numpy()

        on_xs = concatenate(on_xs, on_adv_filtered)
        on_ys = concatenate(on_ys, on_adv_filtered_label)
        on_preds = concatenate(on_preds, pred.detach().cpu().numpy())

    final_loader = get_dataloader(on_xs, on_ys)

    if filter_:

        filtered_onxs = None
        filtered_onys = None

        print('organizing training data..')

        for x_tilde, y in final_loader:
            out = classifier(x_tilde.to(device))
            f_x = F.softmax(out, dim=1)
            _, predicted = f_x.max(1)
            idxs = ~predicted.eq(y.to(device))

            filtered_onxs = concatenate(filtered_onxs, x_tilde[idxs].detach().numpy())
            filtered_onys = concatenate(filtered_onys, y[idxs].detach().numpy())

        total_X = concatenate(orig_X, on_xs)
        total_y = concatenate(orig_y, on_ys)

        final_loader = get_dataloader(filtered_onxs, filtered_onys)

    return final_loader


def analysis_per_sparsity(model_type, cuda_num, data_type, device, train_type, loader, test_loader, EPOCHs=10,
                          retrain_models=False, onadv_loader=None, onadv_test_loader=None, offadv_test_loader=None,
                          sparsity_ratio=0.9, epsilon=1.0, alpha=0.1, k=7):
    assert train_type in ['base', 'robust_train', 'original_robust',
                          'onoff', 'trades'], "train type must be one of ['base', 'robust', 'original_robust', 'onoff']"

    clf = ClassifierTrainer(model_type, cuda_num, loader, test_loader)
    regularize = True
    if sparsity_ratio == 0.:
        regularize = False
        
    orig_file = Path('./simulation/%s/base_%s.pt' % (data_type, model_type))
#     if train_type == 'base' and sparsity_ratio == 0:
#         clf_state = torch.load(orig_file)
#         clf.classifier.load_state_dict(clf_state)
#         classifier = clf.classifier
#     else:
    my_file = Path('./simulation/%s/%s_v2_%s_sparsity%s.pt' % (
        data_type, train_type, model_type, str(sparsity_ratio).replace('.', '_')))
    
    if sparsity_ratio == 0 and train_type == 'base':
        my_file = orig_file
        
    print(my_file)

    if my_file.exists() and not retrain_models:
        print('loading ', my_file)
        clf_state = torch.load(my_file)
        clf.classifier.load_state_dict(clf_state)
        classifier = clf.classifier
    else:
        clf.classifier.load_state_dict(torch.load(orig_file))
        clf.classifier.to(device)
        clf.EPOCH = EPOCHs
        if train_type == 'onoff':
            clf.train(regularize=regularize, ratio=sparsity_ratio, onoff=True, onmanifold_loader=onadv_loader)
        elif train_type == 'original_robust':
            clf.train(regularize=regularize, ratio=sparsity_ratio, AT=True, epsilon=epsilon, alpha=alpha, k=k)
#             elif train_type == 'robust':
#                 clf.train(regularize=regularize, ratio=sparsity_ratio, epsilon=0.3, alpha=0.073, k=7)
        else:
            clf.train(regularize=regularize, ratio=sparsity_ratio)

        classifier = clf.classifier
        torch.save(classifier.cpu().state_dict(), my_file)
    classifier.to(device)
    
#     offadv_test_loader
    
    benign_acc, benign_correct, benign_total = accuracy2(classifier, test_loader, device)
    off_acc, off_correct, off_total = accuracy2(classifier, offadv_test_loader, device)
#     off_acc, off_correct, off_total = accuracy3(classifier, test_loader, device, epsilon=epsilon, alpha=alpha, k=k)
    on_acc, on_correct, on_total = accuracy2(classifier, onadv_test_loader, device)
    
#     normal_acc = (benign_correct + on_correct) / (benign_total + on_total)
    normal_acc = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    
    return benign_acc, off_acc, on_acc, normal_acc, benign_correct, on_correct, benign_total, on_total

#     acc, adv_acc, _, _, advs, benign_softmax_list, adversarial_softmax_list, benign_softmax_list2, adversarial_softmax_list2 = accuracy(classifier, test_loader, device,
#                                                                                        epsilon=epsilon, alpha=alpha,
#                                                                                        k=k)
    
#     onadv_acc, _, _, _, _, onadv_softmax_list, _, onadv_softmax_list2, _ = accuracy(classifier, onadv_test_loader, device,epsilon=epsilon, alpha=alpha, k=k)
#     return acc, adv_acc, onadv_acc, advs, benign_softmax_list, adversarial_softmax_list, onadv_softmax_list, benign_softmax_list2, adversarial_softmax_list2, onadv_softmax_list2


def run_sparsity(model_type, cuda_num, data_type, sparsity_ratio, device, retrain_models = True, EPOCHs = 10, epsilon=1.0, alpha=0.01, k=7):
    
    data_loader, test_loader, _ = mnist(128)
    ## LOAD ALL
    state_dict = load_checkpoint(base_path='./simulation/results', dataset_name='mnist', net_name='vae',
                                 is_best=False, filename='chkpoint_epoch%d.pt' % 14)
#     trainer = VAETrainer(data_loader, cuda_num=2)
#     trainer.gan.load_state_dict(state_dict)
#     gan = trainer.gan

    l_clf = LatentClf()
    l_clf.load_state_dict(torch.load('./simulation/%s/mnist_l_clf.pt' % data_type))

#     e_, k_, alpha_ = 0.3, 10, 0.073
#     datafile = Path('./simulation/%s/onmanifold_%s_%s_k%se%s_loader.pkl'%(data_type, data_type, model_type, str(k_),str(e_).replace('.','_')))
    datafile = Path('./simulation/%s/onmanifold_MNIST_lenet_loader.pkl'%data_type)
    onadv_test_loader = load_data('./simulation/%s/onmanifold_test_%s_%s_loader.pkl'%(data_type, data_type, model_type))
    offadv_test_loader = load_data('./simulation/%s/offmanifold_test_%s_%s_loader.pkl'%(data_type, data_type, model_type))
    onmanifold_train_loader = load_data(datafile)
    
    acc_result = {
        'normal_acc': None,
        'normal_adv_acc': None,
        'normal_onadv_acc': None,
        'normal_onbenign_acc': None,
        'normal_benign_correct': None,
        'normal_benign_total': None,
        'normal_on_correct': None,
        'normal_on_total': None,
        'on_acc': None,
        'on_adv_acc': None,
        'on_onadv_acc': None,
        'on_onbenign_acc': None,
        'on_benign_correct': None,
        'on_benign_total': None,
        'on_on_correct': None,
        'on_on_total': None,
        'off_acc': None,
        'off_adv_acc': None,
        'off_onadv_acc': None,
        'off_onbenign_acc': None,
        'off_benign_correct': None,
        'off_benign_total': None,
        'off_on_correct': None,
        'off_on_total': None,
        'onoff_acc': None,
        'onoff_adv_acc': None,
        'onoff_onadv_acc': None,
        'onoff_onbenign_acc': None,
        'onoff_benign_correct': None,
        'onoff_benign_total': None,
        'onoff_on_correct': None,
        'onoff_on_total': None,
    }

#     softmax_result = {
#         'normal_benign_softmax': None,
#         'on_benign_softmax': None,
#         'off_benign_softmax': None,
#         'onoff_benign_softmax': None,
        
#         'normal_adv_softmax': None,
#         'on_adv_softmax': None,
#         'off_adv_softmax': None,
#         'onoff_adv_softmax': None,
        
#         'normal_onadv_softmax': None,
#         'on_onadv_softmax': None,
#         'off_onadv_softmax': None,
#         'onoff_onadv_softmax': None,
#     }
    
#     total_softmax_result = {
#         'normal_benign_softmax': None,
#         'on_benign_softmax': None,
#         'off_benign_softmax': None,
#         'onoff_benign_softmax': None,
        
#         'normal_adv_softmax': None,
#         'on_adv_softmax': None,
#         'off_adv_softmax': None,
#         'onoff_adv_softmax': None,
        
#         'normal_onadv_softmax': None,
#         'on_onadv_softmax': None,
#         'off_onadv_softmax': None,
#         'onoff_onadv_softmax': None,
#     }

#     adv_data = {
#         'normal_adv_lists': None,
#         'on_adv_lists': None,
#         'off_adv_lists': None,
#         'onoff_adv_lists': None
#     }

    # normal
#     acc_result['normal_acc'], acc_result['normal_adv_acc'], acc_result['normal_onadv_acc']
    acc_result['normal_acc'], acc_result['normal_adv_acc'], acc_result['normal_onadv_acc'], acc_result['normal_onbenign_acc'], acc_result['normal_benign_correct'],acc_result['normal_on_correct'],acc_result['normal_benign_total'],acc_result['normal_on_total'] = analysis_per_sparsity(model_type, cuda_num, data_type, device,
                                                                               'base', data_loader, test_loader, EPOCHs=EPOCHs, onadv_test_loader=onadv_test_loader, offadv_test_loader=offadv_test_loader,
                                                                               retrain_models=retrain_models, sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)
    
    acc_result['on_acc'], acc_result['on_adv_acc'], acc_result['on_onadv_acc'], acc_result['on_onbenign_acc'], acc_result['on_benign_correct'],acc_result['on_on_correct'],acc_result['on_benign_total'],acc_result['on_on_total']  = analysis_per_sparsity(model_type, cuda_num, data_type, device,
                                            'robust_train', onmanifold_train_loader, test_loader, EPOCHs=EPOCHs, onadv_test_loader=onadv_test_loader, offadv_test_loader=offadv_test_loader,
                                            retrain_models=retrain_models, sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)
    
    acc_result['off_acc'], acc_result['off_adv_acc'], acc_result['off_onadv_acc'], acc_result['off_onbenign_acc'], acc_result['off_benign_correct'],acc_result['off_on_correct'],acc_result['off_benign_total'],acc_result['off_on_total'] = analysis_per_sparsity(model_type, cuda_num, data_type, device,
                                                       'original_robust', data_loader, test_loader, EPOCHs=EPOCHs,
                                                       retrain_models=retrain_models, onadv_test_loader=onadv_test_loader, offadv_test_loader=offadv_test_loader,
                                                       sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)
    
    acc_result['onoff_acc'], acc_result['onoff_adv_acc'], acc_result['onoff_onadv_acc'], acc_result['onoff_onbenign_acc'],acc_result['onoff_benign_correct'],acc_result['onoff_on_correct'],acc_result['onoff_benign_total'],acc_result['onoff_on_total'] = analysis_per_sparsity(model_type, cuda_num, data_type, device,
                                                           'onoff', data_loader, test_loader, EPOCHs=EPOCHs,
                                                           retrain_models=retrain_models, onadv_test_loader=onadv_test_loader, offadv_test_loader=offadv_test_loader,
                                                           onadv_loader=onmanifold_train_loader,
                                                           sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)
    
    return acc_result
#     acc_result['normal_acc'], acc_result['normal_adv_acc'], acc_result['normal_onadv_acc'], adv_data['normal_adv_lists'], softmax_result['normal_benign_softmax'], \
#     softmax_result['normal_adv_softmax'], softmax_result['normal_onadv_softmax'], total_softmax_result['normal_benign_softmax'], \
#     total_softmax_result['normal_adv_softmax'], total_softmax_result['normal_onadv_softmax'] = analysis_per_sparsity(model_type, cuda_num, data_type, device,
#                                                            'base', data_loader, test_loader, EPOCHs=EPOCHs, onadv_test_loader=onadv_test_loader,
#                                                            retrain_models=retrain_models, sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)

#     # on AT
#     acc_result['on_acc'], acc_result['on_adv_acc'], acc_result['on_onadv_acc'], adv_data['on_adv_lists'], softmax_result['on_benign_softmax'], \
#     softmax_result['on_adv_softmax'], softmax_result['on_onadv_softmax'], total_softmax_result['on_benign_softmax'], \
#     total_softmax_result['on_adv_softmax'], total_softmax_result['on_onadv_softmax'] = analysis_per_sparsity(model_type, cuda_num, data_type, device,
#                                                 'robust_train', onmanifold_train_loader, test_loader, EPOCHs=EPOCHs, onadv_test_loader=onadv_test_loader,
#                                                 retrain_models=retrain_models, sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)

#     # off AT
#     acc_result['off_acc'], acc_result['off_adv_acc'],acc_result['off_onadv_acc'], adv_data['off_adv_lists'], softmax_result['off_benign_softmax'], \
#     softmax_result['off_adv_softmax'], softmax_result['off_onadv_softmax'], total_softmax_result['off_benign_softmax'], \
#     total_softmax_result['off_adv_softmax'], total_softmax_result['off_onadv_softmax'] = analysis_per_sparsity(model_type, cuda_num, data_type, device,
#                                                            'original_robust', data_loader, test_loader, EPOCHs=EPOCHs,
#                                                            retrain_models=retrain_models, onadv_test_loader=onadv_test_loader,
#                                                            sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)

#     # on + off AT
#     acc_result['onoff_acc'], acc_result['onoff_adv_acc'], acc_result['onoff_onadv_acc'], adv_data['onoff_adv_lists'], softmax_result['onoff_benign_softmax'], \
#     softmax_result['onoff_adv_softmax'], softmax_result['onoff_onadv_softmax'], total_softmax_result['onoff_benign_softmax'], \
#     total_softmax_result['onoff_adv_softmax'], total_softmax_result['onoff_onadv_softmax']= analysis_per_sparsity(model_type, cuda_num, data_type, device,
#                                                            'onoff', data_loader, test_loader, EPOCHs=EPOCHs,
#                                                            retrain_models=retrain_models, onadv_test_loader=onadv_test_loader,
#                                                            onadv_loader=onmanifold_train_loader,
#                                                            sparsity_ratio=sparsity_ratio, epsilon=epsilon, alpha=alpha, k=k)
    
#     return acc_result, softmax_result, total_softmax_result, adv_data

In [ ]:
gpu_idx = 4
device= 'cuda:%d' %gpu_idx

data_type ='MNIST'
model_type='lenet'

data_loader, test_loader, _ = mnist(128)

orig_file = Path('./simulation/%s/base_%s.pt' % (data_type, model_type))
args = {'epsilon': 1.0, 'alpha': 0.01, 'k': 7}

clf = ClassifierTrainer(model_type, gpu_idx, data_loader, test_loader, lr=0.0005)
clf.classifier.load_state_dict(torch.load(orig_file))
clf.classifier.to(device)

classifier = clf.classifier

# offadv_test_loader, advs, correct_ys, offadv_total_loader = generate_adv(classifier, test_loader, device, args['epsilon'], args['alpha'], args['k'])

In [ ]:
offadv_test_loader = load_data('./simulation/%s/offmanifold_test_%s_%s_loader.pkl'%(data_type, data_type, model_type))

In [ ]:
accuracy2(classifier, offadv_test_loader, device)

In [ ]:
# load_data('./simulation/%s/onmanifold_test_%s_%s_loader.pkl'%(data_type, data_type, model_type))
# save_data(offadv_test_loader, './simulation/%s/'%(data_type), 'offmanifold_test_%s_%s_loader.pkl'%(data_type, model_type))

In [ ]:
# sparsity_ratio_list = [] # 0.1, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 0.91, 0.93, 0.95, 0.97, 0.99, 0.993, 0.996, 0.999
sparsity_ratio_list = [0, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 0.995, 0.999]

return_list = []

gpu_idx = 1
device= 'cuda:%d' %gpu_idx

param = {'model_type': 'lenet', 'data_type': 'MNIST', 'retrain_models': False, 'EPOCHs': 15, 'epsilon': 1.0, 'alpha': 0.01, 'k': 7}
for sparsity_ratio in sparsity_ratio_list:
#     , softmax_result, total_softmax_result, adv_data
    acc_result = run_sparsity(param['model_type'], gpu_idx, param['data_type'],
                                                        sparsity_ratio, device, retrain_models=param['retrain_models'], EPOCHs=param['EPOCHs'], epsilon=param['epsilon'],
                                                        alpha=param['alpha'], k=param['k'])
    
#     return_list[sparsity_ratio] = {'acc_result': acc_result, 'softmax_result': softmax_result, 'total_softmax_result': total_softmax_result,
#                                             'adv_data': adv_data}

    return_list.append(acc_result)



In [ ]:
save_data(return_list, './simulation/MNIST/', 'mnist_lenet_normal_rerun.pkl')